In [1]:
import sys
sys.path.append('../')
import os
os.chdir('../')
%reload_ext autoreload
%autoreload 2

import os
import json

import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

import models.vqvae as vqvae
import utils.losses as losses 
import options.option_vq as option_vq
import utils.utils_model as utils_model
from dataset import dataset_VQ, dataset_TM_eval
import utils.eval_trans as eval_trans
from options.get_eval_option import get_opt
from models.evaluator_wrapper import EvaluatorModelWrapper
import warnings
warnings.filterwarnings('ignore')
from utils.word_vectorizer import WordVectorizer

In [2]:
class Temp:
    def __init__(self):
        print('mock:: opt')
args = Temp()
args.dataname = args.dataset_name = 'kit'
w_vectorizer = WordVectorizer('./glove', 'our_vab')

if args.dataname == 'kit' : 
    dataset_opt_path = './checkpoints/kit/Comp_v6_KLD005/opt.txt'  
    args.nb_joints = 21
else :
    dataset_opt_path = './checkpoints/t2m/Comp_v6_KLD005/opt.txt'
    args.nb_joints = 22
wrapper_opt = get_opt(dataset_opt_path, torch.device('cuda'))
eval_wrapper = EvaluatorModelWrapper(wrapper_opt)

mock:: opt
Reading ./checkpoints/kit/Comp_v6_KLD005/opt.txt
Loading Evaluation Model Wrapper (Epoch 30) Completed!!


In [3]:
args.batch_size = 256
args.down_t = 2
args.window_size = 64
train_loader = dataset_VQ.DATALoader(args.dataname,
                                        args.batch_size,
                                        window_size=args.window_size,
                                        unit_length=2**args.down_t)
train_loader_iter = dataset_VQ.cycle(train_loader)

100%|██████████| 4888/4888 [00:01<00:00, 3428.26it/s]

Total number of motions 3122


In [4]:
args.nb_code = 512 # 8192 # 
args.code_dim = 512 # 32 # 
args.output_emb_width = 512
args.down_t = 2
args.stride_t = 2
args.width = 512
args.depth = 3
args.dilation_growth_rate = 3
args.vq_act = 'relu'
args.vq_norm = None

args.quantizer = 'ema_reset'
args.mu = 0.99
net = vqvae.HumanVQVAE(args, ## use args to define different parameters in different quantizers
                       args.nb_code,
                       args.code_dim,
                       args.output_emb_width,
                       args.down_t,
                       args.stride_t,
                       args.width,
                       args.depth,
                       args.dilation_growth_rate,
                       args.vq_act,
                       args.vq_norm)
# net = torch.nn.DataParallel(net)
net.train()
net.cuda()

args.recons_loss = 'l1_smooth'
args.nb_joints
Loss = losses.ReConsLoss(args.recons_loss, args.nb_joints)

In [9]:
#### DEBUG 
# temp = torch.rand(256, 64, 251).cuda()
# pred_motion, loss_commit, perplexity = net(temp)
# pred_motion, loss_commit, perplexity = net(gt_motion)
# print(pred_motion.shape, loss_commit.shape, perplexity.shape)

# net = vqvae.HumanVQVAE(args, ## use args to define different parameters in different quantizers
#                        args.nb_code,
#                        args.code_dim,
#                        args.output_emb_width,
#                        args.down_t,
#                        args.stride_t,
#                        args.width,
#                        args.depth,
#                        args.dilation_growth_rate,
#                        args.vq_act,
#                        args.vq_norm)
# net.cuda()
# from torchsummary import summary
# summary(net.vqvae.encoder, (251, 64))

conv = torch.nn.Conv1d(in_channels=251, out_channels=512, 
                       kernel_size=3, 
                       stride=1, 
                       padding=0)
conv(torch.rand(2, 251, 64)).shape


a torch.Size([2, 251, 64])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
              PrMo-1              [-1, 251, 64]               0
            Conv1d-2              [-1, 512, 64]         386,048
              ReLU-3              [-1, 512, 64]               0
            Conv1d-4              [-1, 512, 32]       1,049,088
          Identity-5              [-1, 512, 32]               0
              ReLU-6              [-1, 512, 32]               0
            Conv1d-7              [-1, 512, 32]         786,944
          Identity-8              [-1, 512, 32]               0
              ReLU-9              [-1, 512, 32]               0
           Conv1d-10              [-1, 512, 32]         262,656
   ResConv1DBlock-11              [-1, 512, 32]               0
         Identity-12              [-1, 512, 32]               0
             ReLU-13              [-1, 512, 32]               0
           C

torch.Size([2, 512, 62])

In [8]:
gt_motion = next(train_loader_iter)
gt_motion.shape

torch.Size([256, 64, 251])

In [13]:
args.commit = 0.02
args.loss_vel = 0.5

gt_motion = next(train_loader_iter)
gt_motion = gt_motion.cuda().float() # (bs, 64, dim)

pred_motion, loss_commit, perplexity = net(gt_motion)
loss_motion = Loss(pred_motion, gt_motion)
loss_vel = Loss.forward_vel(pred_motion, gt_motion)

loss = loss_motion + args.commit * loss_commit + args.loss_vel * loss_vel

### Eval

In [ ]:
val_loader = dataset_TM_eval.DATALoader(args.dataname, False,
                                        32,
                                        w_vectorizer,
                                        unit_length=2**args.down_t)

In [ ]:
class LoggerWriterMock:
    def __init__(self):
        self.info
    def info(self, *args):
        print(*args)
    def add_scalar(self, *args):
        print(*args)
logger = LoggerWriterMock()
logger.info('test')
writer = LoggerWriterMock()
writer.add_scalar('./Test/FID', 'fid', 'nb_iter')

In [ ]:
args.out_dir = './'
a = eval_trans.evaluation_vqvae(
    args.out_dir, 
    val_loader, 
    net, 
    logger, 
    writer, 
    nb_iter = 1, 
    best_fid = 1, 
    best_iter = 1, 
    best_div = 1, 
    best_top1 = 1, 
    best_top2 = 1, 
    best_top3 = 1, 
    best_matching = 1, 
    eval_wrapper=eval_wrapper)


In [ ]:
word_embeddings, pos_one_hots, caption, sent_len, motion, m_length, token, name = next(iter(val_loader))
motion.shape